## Imports

In [ ]:
import os, sys, re
import json, joblib
import ipywidgets as widgets
import papermill as pm
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np 
from IPython.display import clear_output    
lib_path = './../Sources'
if (lib_path not in sys.path):
    sys.path.append(lib_path) #src directory
from lpsrec.utils import create_gif    

In [ ]:
output_folder = './Output_Notebooks'
if not os.path.exists(output_folder):
    print ('Creating folder ' + output_folder)
    os.makedirs(output_folder)

## Parameters

In [ ]:
# Parameters to be used with Papermill
model_tag = 'ItemKNN'
arr_dataset_tag = ['ML1M']
dataset_tag = arr_dataset_tag[0]
evaluation_metrics = ['PREC', 'RECALL', 'NDCG', 'MRR', 'MAP']
rank_length = 30
nodes = 10
n_folds = 5

In [ ]:
# for dataset_tag in arr_dataset_tag:
#     # Parallelize here later    
#     for partition in [x+1 for x in range(nodes)]:
#         try:            
#             print ("Starting partition {}".format(partition))            
#             pm.execute_notebook(
#                './MostPopular Case Recommender Eval on Sparsity Datasets.ipynb',
#                 os.path.join(output_folder, 'MostPopular Case Recommender Eval on Sparsity Datasets [{}][{}].ipynb'.format(dataset_tag, partition)),
#                 parameters = dict(
#                     dataset_tag = dataset_tag,
#                     model_tag = 'MostPopular',
#                     rank_length = 20,
#                     random_state = 31415,
#                     evaluation_metrics = evaluation_metrics,
#                     bot_alive = True,
#                     partition = partition,
#                     nodes = 5)
#             )
#         except Exception as e:
#             print ("Error running dataset {} on partition {}: {}".format(dataset_tag, partition, e))

## Plotting Results

In [ ]:
style_dict = json.load(open('./style_dict.json', 'r'))
plt.rc('font', **style_dict['font'])
# plt.rc('axes.titlesize', fontsize=20)
plt.rc('xtick', labelsize=style_dict['tick']['fontsize']) 
plt.rc('ytick', labelsize=style_dict['tick']['fontsize']) 
plt.rcParams.update({'figure.max_open_warning': 5})

In [ ]:
bot_alive = False
dataset_output_folder = os.path.join('.', 'Outputs', dataset_tag)

Grouping all folders eval metadata into a single file.

The output should be a df_eval_metadata_fold{} containing all metrics and all sparsity cenarios

In [ ]:
for n_fold in np.arange(1, n_folds+1):
    for partition in np.arange(1, nodes+1, 1):
        analysis_tag = '_'.join([str(x) for x in [rank_length, nodes, partition]])
        variables_output_folder = os.path.join(dataset_output_folder, model_tag, 'Variables', analysis_tag)
        figures_output_folder = os.path.join(dataset_output_folder, model_tag, 'Figures', analysis_tag)
        df_eval_metadata_temp = pd.read_csv(os.path.join(variables_output_folder, 'df_eval_metadata_fold{}_{}.tsv'.format(n_folds, n_fold)), sep = '\t', header = 0)    
        df_eval_metadata = df_eval_metadata_temp.copy() if partition == 1 else df_eval_metadata.append(df_eval_metadata_temp)        
    df_eval_metadata.reset_index(drop = True, inplace = True)
    analysis_tag = '_'.join([str(x) for x in [rank_length, 'None']])
    variables_output_folder = os.path.join(dataset_output_folder, model_tag, 'Variables', analysis_tag)
    figures_output_folder = os.path.join(dataset_output_folder, model_tag, 'Figures', analysis_tag)
    if not os.path.exists(variables_output_folder):    
        os.makedirs(variables_output_folder)    
    if not os.path.exists(figures_output_folder):        
        os.makedirs(figures_output_folder)
    
    df_eval_metadata.to_csv(os.path.join(variables_output_folder, 'df_eval_metadata_fold{}.tsv'.format(n_fold)), sep = '\t', header = True, index = False)    
del df_eval_metadata_temp
del df_eval_metadata

Now that we have a df_eval_metadata_fold{}, we can create an arr_metrics for each metrics in each folder and append each arr_metric to an arr_metrics_folder.

Example: for precision@k

    arr_metric_folds = [arr_metric_fold1, arr_metric_fold2, ...] 
    

In [ ]:
dict_df_eval_metadata = dict()
for n_fold in np.arange(1, n_folds+1):
    filename = os.path.join(variables_output_folder, 'df_eval_metadata_fold{}.tsv'.format(n_fold))
    print (">> Getting file ", filename)
    df_eval_metadata = pd.read_csv(filename, sep = '\t', header = 0) 
    dict_df_eval_metadata[n_fold] = df_eval_metadata

In [ ]:
%%time
uss_limits = np.sort(np.array(df_eval_metadata['uss'].unique()))
iss_limits = np.sort(np.array(df_eval_metadata['iss'].unique()))
rank_lengths = np.arange(1, rank_length+1, 1) # Setting extra rank analysis
arr_metrics = np.repeat(None, n_folds)
  
for rank in rank_lengths: # [1, 2, 3, ..., rank_length]     
    dict_arr_metrics = dict()
    for m in evaluation_metrics:
        dict_arr_metrics[m] = np.repeat(None, n_folds)
    arr_rank_metrics = [m + '@' + str(rank) for m in evaluation_metrics]
    
    for column in arr_rank_metrics: # [PREC@1, RECALL@1, NDCG@1, ...]
        print ("\n> Processing ", column)
        arr_metric = np.zeros([len(uss_limits), len(iss_limits)])
        for n_fold in np.arange(1, n_folds+1):                        
            for uss_index, uss_limit in enumerate(uss_limits):
                for iss_index, iss_limit in enumerate(iss_limits):            
                    arr_metric[uss_index, iss_index] = dict_df_eval_metadata[n_fold][(dict_df_eval_metadata[n_fold]['uss'] == uss_limit) & (dict_df_eval_metadata[n_fold]['iss'] == iss_limit)][column].reset_index(drop = True)[0]
            # Colocar em cada metrica
            dict_arr_metrics[re.split("@", column)[0]][n_fold-1] = arr_metric
        filename = os.path.join(variables_output_folder, 'dict_arr_metrics_' + column.lower() + '_' + model_tag + '.joblib')
        print ("> Writing file ", filename)
        joblib.dump(dict_arr_metrics, filename)        
    
    # Elapsed time analysis
#     arr_metric = np.zeros([len(uss_limits), len(iss_limits)])
#     for uss_index, uss_limit in enumerate(uss_limits):
#         for iss_index, iss_limit in enumerate(iss_limits):            
#             arr_metric[uss_index, iss_index] = df_eval_metadata[(df_eval_metadata['uss'] == uss_limit) & (df_eval_metadata['iss'] == iss_limit)]['elapsed_time'].reset_index(drop = True)[0]
#     joblib.dump(arr_metric, os.path.join(variables_output_folder, 'arr_elapsed_time_' + model_tag + '.joblib'))        

Plotting mean of all metrics 

In [ ]:
cmapping = "jet"
tick_step = 5
figs = {'mean': {}, 'std': {}}
for metric in evaluation_metrics:
    figs['mean'][metric+'@'] = list()

for rank in rank_lengths:#[-1:-2:-1]:
    arr_rank_metrics = [m + '@' + str(rank) for m in evaluation_metrics]
    for column in arr_rank_metrics:
        dict_arr_metrics = joblib.load(os.path.join(variables_output_folder, 'dict_arr_metrics_' + column.lower() + '_' + model_tag + '.joblib'))        
        arr_metric = np.mean(dict_arr_metrics[re.split("@", column)[0]], axis=0)

        fig, ax = plt.subplots(figsize=(style_dict['figure']['width'],style_dict['figure']['height']))    
        cax = plt.imshow(arr_metric, cmap=cmapping)
        plt.gca().invert_yaxis()
        cbar = plt.colorbar(cax, ticks = [x/100.0 for x in np.arange(0,1000,10)], shrink = 0.83)

        ax.set_xticklabels(uss_limits[0:len(uss_limits):tick_step])
        ax.set_yticklabels(iss_limits[0:len(uss_limits):tick_step])
        ax.set_xticks(np.arange(0, len(uss_limits), tick_step))    
        ax.set_yticks(np.arange(0, len(iss_limits), tick_step))    
        ax.set_xlabel('Last User Specific Sparsity', fontsize = style_dict['label']['fontsize'])
        ax.set_ylabel('Last Item Specific Sparsity', fontsize = style_dict['label']['fontsize'])    
        ax.tick_params(axis='both', which='major', labelsize=style_dict['tick']['fontsize'])
        cbar.set_label(column.title(), labelpad=-50,  y=1.08, rotation=0, fontsize = style_dict['label']['fontsize'])
        cbar.ax.tick_params(labelsize = style_dict['tick']['fontsize'])
        plt.clim(0, 1)
        plt.xticks(rotation = 'vertical')

        filename = '2d-' + column + '_mean.png'       
#         fig.savefig(os.path.join(figures_output_folder, filename), bbox_inches = 'tight')
        figs['mean'][re.split("\d", column)[0]].append(fig) 
        if rank == rank_length: # Send only the target-analysis
            bot.send_message(filePath=os.path.join(figures_output_folder, filename)) if bot_alive else ''
        plt.close()

Plotting std of all metrics

In [ ]:
cmapping = "jet"
tick_step = 5
for metric in evaluation_metrics:
    figs['std'][metric+'@'] = list()
    
for rank in rank_lengths:
    arr_rank_metrics = [m + '@' + str(rank) for m in evaluation_metrics]
    for column in arr_rank_metrics:
        dict_arr_metrics = joblib.load(os.path.join(variables_output_folder, 'dict_arr_metrics_' + column.lower() + '_' + model_tag + '.joblib'))        
        arr_metric = np.std(dict_arr_metrics[re.split("@", column)[0]], axis=0)

        fig, ax = plt.subplots(figsize=(style_dict['figure']['width'],style_dict['figure']['height']))    
        cax = plt.imshow(arr_metric, cmap=cmapping)
        plt.gca().invert_yaxis()
        cbar = plt.colorbar(cax, shrink = 0.83)

        ax.set_xticklabels(uss_limits[0:len(uss_limits):tick_step])
        ax.set_yticklabels(iss_limits[0:len(uss_limits):tick_step])
        ax.set_xticks(np.arange(0, len(uss_limits), tick_step))    
        ax.set_yticks(np.arange(0, len(iss_limits), tick_step))    
        ax.set_xlabel('Last User Specific Sparsity', fontsize = style_dict['label']['fontsize'])
        ax.set_ylabel('Last Item Specific Sparsity', fontsize = style_dict['label']['fontsize'])    
        ax.tick_params(axis='both', which='major', labelsize=style_dict['tick']['fontsize'])
        cbar.set_label(column.title(), labelpad=-50,  y=1.08, rotation=0, fontsize = style_dict['label']['fontsize'])
        cbar.ax.tick_params(labelsize = style_dict['tick']['fontsize'])
        plt.clim(0, np.max(np.std(dict_arr_metrics[re.split("@", column)[0]], axis=0)))
        plt.xticks(rotation = 'vertical')

        filename = '2d-' + column + '_std.png'       
        fig.savefig(os.path.join(figures_output_folder, filename), bbox_inches = 'tight')
        figs['std'][re.split("\d", column)[0]].append(fig) 
        if rank == rank_length: # Send only the target-analysis
            bot.send_message(filePath=os.path.join(figures_output_folder, filename)) if bot_alive else ''
        plt.close()

In [ ]:
for metric in evaluation_metrics:    
    filepaths = [os.path.join(figures_output_folder, '2d-' + column + '_mean.png' ) for column in [metric + '@' + str(rank) for rank in rank_lengths]]
    output_filepath = os.path.join(figures_output_folder, '2d-' + metric + '@k_mean' + '.gif' )
    create_gif(filepaths, output_filepath, duration=0.2)

In [ ]:
for metric in evaluation_metrics:    
    filepaths = [os.path.join(figures_output_folder, '2d-' + column + '_std.png' ) for column in [metric + '@' + str(rank) for rank in rank_lengths]]
    output_filepath = os.path.join(figures_output_folder, '2d-' + metric + '@k_std' + '.gif' )
    create_gif(filepaths, output_filepath, duration=0.2)

In [ ]:
show_metric = lambda metric, k, stat: figs[stat][metric + '@'][int(k)-1]
widgets.interact(show_metric, stat=['mean', 'std'], k=widgets.IntSlider(min=1, max=rank_length, step=1, value=10), figs=figs, metric=evaluation_metrics)

### Running Time

In [ ]:
df_eval_metadata.head()

In [ ]:
df_time_uss = df_eval_metadata[['uss', 'elapsed_time']].groupby(['uss']).mean().reset_index(drop = False)
df_time_iss = df_eval_metadata[['iss', 'elapsed_time']].groupby(['iss']).mean().reset_index(drop = False)
df_time_uss.head()

In [ ]:
fig, ax = plt.subplots(figsize=(style_dict['figure']['width'],style_dict['figure']['height']))    
ax.plot(df_time_uss['uss'], df_time_uss['elapsed_time'], label='USS')
ax.plot(df_time_iss['iss'], df_time_uss['elapsed_time'], label='ISS')
ax.set_ylabel('Elapsed Time (s)', fontsize = style_dict['label']['fontsize'])
ax.grid(True)
ax.legend()
filename = '2d-elapsed_time_mean_uss_iss.png'       
fig.savefig(os.path.join(figures_output_folder, filename), bbox_inches = 'tight')

In [ ]:
arr_metric = joblib.load(os.path.join(variables_output_folder, 'arr_elapsed_time_' + model_tag + '.joblib'))        

fig, ax = plt.subplots(figsize=(style_dict['figure']['width'],style_dict['figure']['height']))    
cax = plt.imshow(arr_metric, cmap=cmapping)
plt.gca().invert_yaxis()
cbar = plt.colorbar(cax, shrink = 0.83)

ax.annotate('Min: {:.02f}\nMax: {:.02f}'.format(np.min(arr_metric), np.max(arr_metric)), xy = (0, 0), fontsize=style_dict['label']['fontsize']-2, color='white')
ax.set_xticklabels(uss_limits[0:len(uss_limits):tick_step])
ax.set_yticklabels(iss_limits[0:len(uss_limits):tick_step])
ax.set_xticks(np.arange(0, len(uss_limits), tick_step))    
ax.set_yticks(np.arange(0, len(iss_limits), tick_step))    
ax.set_xlabel('Last User Specific Sparsity', fontsize = style_dict['label']['fontsize'])
ax.set_ylabel('Last Item Specific Sparsity', fontsize = style_dict['label']['fontsize'])    
ax.tick_params(axis='both', which='major', labelsize=style_dict['tick']['fontsize'])
cbar.set_label('Elapsed Time (s)', labelpad=-90,  y=1.08, rotation=0, fontsize = style_dict['label']['fontsize'])
cbar.ax.tick_params(labelsize = style_dict['tick']['fontsize'])
plt.xticks(rotation = 'vertical')
plt.clim(np.floor(np.min(arr_metric)), np.ceil(np.max(arr_metric)))
filename = '2d-elapsed_time.png'       
fig.savefig(os.path.join(figures_output_folder, filename), bbox_inches = 'tight')
figs[re.split("\d", column)[0]].append(fig) 
if rank == rank_length: # Send only the target-analysis
    bot.send_message(filePath=os.path.join(figures_output_folder, filename)) if bot_alive else ''
#         clear_output()
# plt.close()

__________________

In [ ]:
# cmapping = "jet"
# tick_step = 5
# figs = dict()
# for metric in evaluation_metrics:
#     figs[metric+'@'] = list()

# for rank in rank_lengths:
#     arr_rank_metrics = [m + '@' + str(rank) for m in evaluation_metrics]
#     for column in arr_rank_metrics[:1]:
#         arr_metric = joblib.load(os.path.join(variables_output_folder, 'arr_' + column.lower() + '_' + model_tag + '.joblib'))        

#         fig, ax = plt.subplots(figsize=(style_dict['figure']['width'],style_dict['figure']['height']))    
#         cax = plt.imshow(arr_metric, cmap=cmapping)
#         plt.gca().invert_yaxis()
#         cbar = plt.colorbar(cax, ticks = [x/100.0 for x in np.arange(0,1000,10)], shrink = 0.83)

#         ax.set_xticklabels(uss_limits[0:len(uss_limits):tick_step])
#         ax.set_yticklabels(iss_limits[0:len(uss_limits):tick_step])
#         ax.set_xticks(np.arange(0, len(uss_limits), tick_step))    
#         ax.set_yticks(np.arange(0, len(iss_limits), tick_step))    
#         ax.set_xlabel('Last User Specific Sparsity', fontsize = style_dict['label']['fontsize'])
#         ax.set_ylabel('Last Item Specific Sparsity', fontsize = style_dict['label']['fontsize'])    
#         ax.tick_params(axis='both', which='major', labelsize=style_dict['tick']['fontsize'])
#         cbar.set_label(column.title(), labelpad=-50,  y=1.08, rotation=0, fontsize = style_dict['label']['fontsize'])
#         cbar.ax.tick_params(labelsize = style_dict['tick']['fontsize'])
#         plt.clim(0, 1)
#         plt.xticks(rotation = 'vertical')

#         filename = '2d-' + column + '.png'       
#         fig.savefig(os.path.join(figures_output_folder, filename), bbox_inches = 'tight')
#         figs[re.split("\d", column)[0]].append(fig) 
#         if rank == rank_length: # Send only the target-analysis
#             bot.send_message(filePath=os.path.join(figures_output_folder, filename)) if bot_alive else ''
# #         clear_output()
#         plt.close()

In [ ]:
print (arr_metric.shape)
arr_metric

In [ ]:
uss_limits

In [ ]:
10*int(np.min(arr_metric)*10)

In [ ]:
[x/100.0 for x in np.arange(10*int(np.min(arr_metric)*10), 101, 10)]

In [ ]:
fig, ax = plt.subplots(figsize=(style_dict['figure']['width'],style_dict['figure']['height']))    
cs = plt.pcolormesh(uss_limits, 
                  iss_limits, 
                  arr_metric, 
                  cmap=cmapping,
#                   linewidths=(0,),
#                   colors='black',
#                   levels=[x/100.0 for x in np.arange(0, 101, 2)]
                   )
# ax.clabel(cs, inline=False, fontsize=20, color='k')
# plt.clim(0, 1)
# cs.cmap.set_over('red')
# cs.cmap.set_under('blue')
fig.colorbar(cs, ax=ax, ticks=[x/100.0 for x in np.arange(0, 101, 10)])
ax.grid(c='k', ls='-', alpha=0.5)
cs.changed()

In [ ]:
fig, ax = plt.subplots(figsize=(style_dict['figure']['width'],style_dict['figure']['height']))    
cax = plt.imshow(arr_metric, cmap=cmapping)
plt.gca().invert_yaxis()
cbar = plt.colorbar(cax, ticks = [x/100.0 for x in np.arange(0,1000,10)], shrink = 0.83)

ax.set_xticklabels(uss_limits[0:len(uss_limits):tick_step])
ax.set_yticklabels(iss_limits[0:len(uss_limits):tick_step])
ax.set_xticks(np.arange(0, len(uss_limits), tick_step))    
ax.set_yticks(np.arange(0, len(iss_limits), tick_step))    
ax.set_xlabel('Last User Specific Sparsity', fontsize = style_dict['label']['fontsize'])
ax.set_ylabel('Last Item Specific Sparsity', fontsize = style_dict['label']['fontsize'])    
ax.tick_params(axis='both', which='major', labelsize=style_dict['tick']['fontsize'])
cbar.set_label(column.title(), labelpad=-50,  y=1.08, rotation=0, fontsize = style_dict['label']['fontsize'])
cbar.ax.tick_params(labelsize = style_dict['tick']['fontsize'])
plt.clim(0, 1)
plt.xticks(rotation = 'vertical')